In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output

In [110]:
TIEMPO_DE_ESPERA = 5.5

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=1825x1444")
# options.headless = True
browser = webdriver.Chrome(options = chrome_options)


In [111]:
browser.get('https://www.forever21.com/us/shop')

In [112]:
#CLICK EN LA PARTE SUPERIOR DERECHA, DONDE HAY UN MODAL DE CONFIGURACIÓN DE PAÍS / MONEDA
browser.execute_script("document.getElementsByClassName('table sameline fr')[0].getElementsByTagName('span')[0].click();")
#ASIGNAR DOLAR 
browser.execute_script("document.getElementsByTagName('select')[1].value ='USD'")
time.sleep(2)
#CLICK EN EL BOTON DE GUARDAR 
browser.find_element_by_xpath('//*[@id="globale_csc_popup"]/div/div/div/div/div[3]/input').click()

In [113]:
url_base = "https://www.forever21.com"

In [114]:
lista_productos = []
#Este funciona
#TOMO TODOS MENOS EL ÚLTIMO, YA QUE ES 'SALE'
ELEMENTS_TOP = browser.find_element_by_class_name('full').find_elements_by_class_name('d_new_mega_menu')[:-1]

for TOP in ELEMENTS_TOP:
    
    #CLOTHING, ES EL SEGUNDO ELEMENTO DE LA MATRIZ, O SEA EL [1]
    HREFS_PROBABLEMENTE = TOP.find_element_by_class_name('d_new_mega_sub').find_element_by_class_name('ws_100').find_elements_by_class_name('pa_20')[1].find_element_by_class_name('pl_20').find_element_by_class_name('vt').find_elements_by_tag_name('p')[1:]
    
    LINKS = [HREF.find_element_by_tag_name('a').get_attribute('href') for HREF in HREFS_PROBABLEMENTE]
    

In [121]:
LINKS = []
ELEMENTS_TOP = browser.execute_script('''
                var x = document.getElementsByClassName('d_new_mega_menu hover_p ').length;
                //le quito dos porque el ultimo es "SALE" , y "MAKE UP/ACC".

                x = x-2;
                return x;
                ''')


for i in range(ELEMENTS_TOP):
    
    TIPOLOGIA_POR_CATEGORIA = browser.execute_script(f'''
                            var x = document.getElementsByClassName('d_new_mega_menu hover_p ')[{i}].getElementsByClassName('pa_20 pt_0 vt bl_m_gray wmx_800')[0].getElementsByClassName('pl_20 pr_20 sameline t_black table')[0].getElementsByClassName('vt ws_60')[0].getElementsByTagName('p').length;
                            return x;
                            ''')
    for tipo in range(1,TIPOLOGIA_POR_CATEGORIA):
        LINKS.append(url_base + browser.execute_script(f'''var x = document.getElementsByClassName('d_new_mega_menu hover_p ')[{i}].getElementsByClassName('pa_20 pt_0 vt bl_m_gray wmx_800')[0].getElementsByClassName('pl_20 pr_20 sameline t_black table')[0].getElementsByClassName('vt ws_60')[0].getElementsByTagName('p')[{tipo}].getElementsByTagName('a')[0].getAttribute('href');
        return x;'''))


In [124]:
dfLinks = pd.DataFrame(LINKS)

In [143]:
WOMEN,MEN,PS,GIRLS = [],[],[],[]
for index,row in dfLinks.iterrows():
    if row[0].find('/f21/') != -1 :
        WOMEN.append(row[0])
    if row[0].find('/21men/') != -1 :
        MEN.append(row[0])
    if row[0].find('/plus_size') != -1 :
        PS.append(row[0])
    if row[0].find('/girls/') != -1 :
        GIRLS.append(row[0])


In [147]:
Z=[]
for w,m,p,g in zip(WOMEN,MEN,PS,GIRLS):
    Z.append([w,m,p,g])
    

In [152]:
LINKS = pd.DataFrame(Z)
LINKS.rename(columns={0:'WOMEN',
                     1:'MEN',
                     2:'PLUS_SIZE',
                     3:'GIRLS'},inplace=True)

In [157]:
LINKS.to_excel('LINKS_F21.xlsx')

In [117]:
browser.save_screenshot('asd.png')

True

In [91]:
for link in LINKS:

    browser.get(link)

    tipologia = browser.find_element_by_xpath('//*[@id="multi_submenu_title"]/h1').text


    max_page = browser.execute_script('''
        var aux = document.getElementsByClassName('fr txr show_desktop s_page')[0].getElementsByClassName('pageno').length;

        var max_page = document.getElementsByClassName('fr txr show_desktop s_page')[0].getElementsByClassName('pageno')[aux-2].textContent;

        return max_page;

        ''')
    max_page = int(max_page)

    for page in range(max_page):

        #cantidad de productos dentro de la página.
    ##  productos = browser.find_element_by_id('products').find_elements_by_class_name('pi_container')
        productos = browser.execute_script("var x = document.getElementsByClassName('qv square').length; return x;")
        if browser.current_url == 'https://www.forever21.com/us/shop':
                break
        for index_articulo in range(productos):
            try:
                #URL = articulo.find_element_by_tag_name('div').find_element_by_tag_name('div').find_element_by_tag_name('a').get_attribute('href')
                URL = url_base + browser.execute_script(f"var x = document.getElementsByClassName('item_slider product_link')[{index_articulo}].getAttribute('href');return x;")
            except:
                continue
        ##  ActionChains(browser).move_to_element(articulo).perform()
            try:
                browser.execute_script(f"document.getElementsByClassName('qv square')[{index_articulo}].click();")
            except:
                continue
            #boton.click()
            if browser.current_url == 'https://www.forever21.com/us/shop':
                break
            #ELEMENT_COLORS = browser.find_element_by_xpath('//*[@id="quickview_pop"]/div/div/div/div[2]/div[2]').find_element_by_id('colorButton').find_elements_by_tag_name('li')
            ###ELEMENT_COLORS = browser.execute_script(f"var x = document.getElementsByClassName('color')[{index_articulo}].getElementsByTagName('li').length; return x")
            time.sleep(.5)
            try:
                ELEMENT_COLORS = browser.execute_script(f"var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li').length; return x;")
            except:
                time.sleep(3.5)
                pass
            try:
                ELEMENT_COLORS = browser.execute_script(f"var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li').length; return x;")
            except:
                continue


            for item in range(ELEMENT_COLORS):

                #browser.execute_script(f"document.getElementsByClassName('color')[{index_articulo}].getElementsByTagName('li')[{item}].getElementsByTagName('span')[0].getElementsByTagName('img')[0].click();")
                time.sleep(.5)
                try:
                    browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                except:
                    try:
                        time.sleep(3.5)
                        browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                    except:
                        break
                time.sleep(.5)
                try:
                    NOMBRE_COLOR = browser.execute_script("var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByTagName('p')[0].textContent; return x;").strip()
                except:
                    time.sleep(3.5)
                    NOMBRE_COLOR = browser.execute_script("var x = document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByTagName('p')[0].textContent; return x;").strip()

                #NOMBRE_COLOR = item.find_element_by_tag_name('span').find_element_by_tag_name('img').get_attribute('alt')
                #item.click()

                TALLES = browser.find_element_by_xpath('//*[@id="sizeButton"]').find_elements_by_tag_name('li')
                for TALLE in TALLES:

                    #TALLE BLOQUEADO ES OOS
                    if TALLE.find_element_by_tag_name('span').get_attribute('class') != "oos":

                        NOMBRE_TALLE = TALLE.find_element_by_tag_name('span').text

                        # IMG PRODUCTO
                        IMG = browser.execute_script(f"var x = document.getElementsByClassName('ws_100 owl-carousel owl-loaded owl-drag')[0].getElementsByTagName('img')[0].getAttribute('src'); return x")
                        #IMG = browser.execute_script(f"var x = document.getElementsByClassName('product_image')[{index_articulo}].getAttribute('src'); return x")
                        # DESCRIPCION
                        DESCRIPCION = browser.execute_script(f"var x = document.getElementsByClassName('p_name')[{index_articulo}].textContent; return x")

                        # PRECIO
                        PRECIO = browser.execute_script("var x = document.getElementsByClassName('p_price pt_5 ws_100 row mb_10')[0].textContent; return x").strip()
                        # CODIGO
                        codigo = browser.execute_script("var x = document.getElementsByClassName('t_mid pt_10')[0].textContent; return x")
                        CODIGO = codigo.split()[-1]

                        lista_productos.append([CODIGO,
                                                NOMBRE_TALLE,
                                                NOMBRE_COLOR,
                                                DESCRIPCION,
                                                PRECIO,
                                                IMG,
                                                URL,
                                                tipologia])

            try:
                browser.execute_script("document.getElementsByClassName('quick_close')[0].click();")
            except:
                pass
        #NEXTPAGE
        time.sleep(.8)
        try:
            browser.execute_script("document.getElementsByClassName('p_next')[0].click();")
        except:
            print(browser.current_url,
                 tipologia,
                 link)
            break

WebDriverException: Message: no such session
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-65-generic x86_64)


In [96]:
LINKS

['https://www.forever21.com/us/shop/catalog/category/f21/activewear',
 'https://www.forever21.com/us/shop/catalog/category/f21/top_bodysuits-bustiers',
 'https://www.forever21.com/us/shop/catalog/category/f21/bottoms',
 'https://www.forever21.com/us/shop/catalog/category/f21/dress',
 'https://www.forever21.com/us/shop/catalog/category/f21/top_graphic-tops',
 'https://www.forever21.com/us/shop/catalog/category/f21/lingerie',
 'https://www.forever21.com/us/shop/catalog/category/f21/outerwear_coats-and-jackets',
 'https://www.forever21.com/us/shop/catalog/category/f21/bottom_jeans',
 'https://www.forever21.com/us/shop/catalog/category/f21/rompers-jumpsuits',
 'https://www.forever21.com/us/shop/catalog/category/f21/sets',
 'https://www.forever21.com/us/shop/catalog/category/f21/sweater',
 'https://www.forever21.com/us/shop/catalog/category/f21/swimwear_all',
 'https://www.forever21.com/us/shop/catalog/category/f21/top_blouses']

In [97]:
link

'https://www.forever21.com/us/shop/catalog/category/f21/top_bodysuits-bustiers'

In [63]:
end_= datetime.datetime.now()

['https://www.forever21.com/us/shop/catalog/category/f21/activewear',
 'https://www.forever21.com/us/shop/catalog/category/f21/top_bodysuits-bustiers',
 'https://www.forever21.com/us/shop/catalog/category/f21/bottoms',
 'https://www.forever21.com/us/shop/catalog/category/f21/dress',
 'https://www.forever21.com/us/shop/catalog/category/f21/top_graphic-tops',
 'https://www.forever21.com/us/shop/catalog/category/f21/lingerie',
 'https://www.forever21.com/us/shop/catalog/category/f21/outerwear_coats-and-jackets',
 'https://www.forever21.com/us/shop/catalog/category/f21/bottom_jeans',
 'https://www.forever21.com/us/shop/catalog/category/f21/rompers-jumpsuits',
 'https://www.forever21.com/us/shop/catalog/category/f21/sets',
 'https://www.forever21.com/us/shop/catalog/category/f21/sweater',
 'https://www.forever21.com/us/shop/catalog/category/f21/swimwear_all',
 'https://www.forever21.com/us/shop/catalog/category/f21/top_blouses']

In [86]:
index_articulo

14

In [75]:
browser.save_screenshot('asd.png')

True

In [13]:
print(browser.current_url)

https://www.forever21.com/us/shop/catalog/category/f21/bottoms


In [78]:
URL

'https://www.forever21.com/us/shop/catalog/product/f21/top_bodysuits-bustiers/2000385725'

In [92]:
len(lista_productos)

2534

In [26]:
max_page

10

In [100]:
browser.save_screenshot('asd.png')

WebDriverException: Message: no such session
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-65-generic x86_64)


In [94]:
len(df.drop_duplicates()),len(df)

(2534, 2534)

In [93]:
df = pd.DataFrame(lista_productos)

df.rename(columns={0:'id',
          1:'talle',
          2:'color',
          3:'descripcion',
          4:'precio',
          5:'url_jpg',
          6:'url',
          7:'tipo'},inplace=True)
df.drop_duplicates()

,id,talle,color,descripcion,precio,url_jpg,url,tipo
0,2000384307,S,WHITE,Active Ribbed Knit Crop Top,$‌15.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
1,2000384307,M,WHITE,Active Ribbed Knit Crop Top,$‌15.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
2,2000384307,L,WHITE,Active Ribbed Knit Crop Top,$‌15.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
3,2000384307,S,BLACK,Active Ribbed Knit Crop Top,$‌15.00,/us/shop/_common/images/c_loading.gif,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
4,2000384307,M,BLACK,Active Ribbed Knit Crop Top,$‌15.00,/us/shop/_common/images/c_loading.gif,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
5,2000384307,L,BLACK,Active Ribbed Knit Crop Top,$‌15.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
6,2000389335,S,BLACK,Active Lasercut Capri Leggings,$‌18.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
7,2000389335,M,BLACK,Active Lasercut Capri Leggings,$‌18.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
8,2000389335,L,BLACK,Active Lasercut Capri Leggings,$‌18.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes
9,2000371202,XS,TAUPE/SILVER,Medium Impact - Pindot Sports Bra,$‌18.00,https://www.forever21.com/images/1_front_750/0...,https://www.forever21.com/us/shop/catalog/prod...,Women's Activewear & Workout Clothes


In [95]:
df.to_excel('f21.xlsx')

In [261]:
browser.save_screenshot('asd.png')

True

In [47]:
import calendar

In [48]:
import holidays